In [5]:
import tabula #Extracting tables from pdf
#from pdfminer.high_level import extract_pages, ectract_text
import os
import re
from tika import parser # pip install tika

import pandas as pd
import numpy as np

In [6]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

In [7]:
path = os.listdir("/Users/Moni/Ironhack/my_ironhack/projects/final-project/data/historico-facturas-pdf/more")

# Extract name of documents
files = [i for i in path if ".DS" not in i]

#for testing
#files = files[3:5]

In [8]:
pwd

'/Users/Moni/Ironhack/my_ironhack/projects/final-project/notebooks/historical-purchase-data'

In [75]:
df = pd.DataFrame(columns=['description', 'units', 'price', "date", "order_num"])

path = "../../data/historico-facturas-pdf/more/"

for pdf in files:
    
    order_num = pdf.split("-")[1].replace(".pdf", "")
    
    #parser the text in the pdf (returns a dict)
    raw = parser.from_file(f"{path}{pdf}")
    
    # Extract the content (returns a str)
    content = raw['content']
    
    # He know there is only one date in the document (and the format)
    pattern = r'\b\d{2}/\d{2}/\d{2}\b'  #meets the date format
    
    # Extract the date - returns a list of one elemnt
    date = re.findall(pattern, content)[0]
    
    #Extract tables from the pdf:
    tables = tabula.read_pdf(f"{path}{pdf}", pages="all", pandas_options={'header': None})
    
    #We know last table is always resume of ticket.exclude:
    #tables = tables[:-1]
    
    #Information about products(main table) in different pages, are extracted as different tables:
    for table in tables:
        
        if table[0][0] == "IVA":
            continue
        
        table.rename(columns={0: "description", 1:"units", 2:"price"}, inplace=True)
        table["date"] = date
        table["order_num"] = order_num

        result = pd.concat([df, table])

        df = result
print("pdf tables extrated")

pdf tables extrated


In [77]:
df

,description,units,price,date,order_num,3
0,Detergente ropa prendas delicadas Bosque Verde...,1,"1,69 €",10/12/20,7119732,NaN
1,Raclette queso Entremont lonchas,1,"3,85 €",10/12/20,7119732,NaN
2,Refresco Coca-Cola,1,"6,96 €",10/12/20,7119732,NaN
3,Queso blando blanco y azul Bergader petit blu,1,"1,85 €",10/12/20,7119732,NaN
4,Queso brie con trufa Castello,1,"2,10 €",10/12/20,7119732,NaN
...,...,...,...,...,...,...
27,Mantequilla sin sal añadida Hacendado,1,"1,65 €",11/07/20,959862,NaN
28,Barra pan de pueblo rebanada,2,"2,30 €",11/07/20,959862,NaN
29,Lomo de pavo Hacendado lonchas,1,"1,89 €",11/07/20,959862,NaN
30,Pechuga de pavo Hacendado finas lonchas,1,"2,95 €",11/07/20,959862,NaN


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1015 entries, 0 to 31
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  988 non-null    object
 1   units        877 non-null    object
 2   price        962 non-null    object
 3   date         1015 non-null   object
 4   order_num    1015 non-null   object
 5   3            89 non-null     object
dtypes: object(6)
memory usage: 55.5+ KB


In [79]:
def drop_na(df, column):
    nans = df[df[column].isna()].shape[0]
    df.dropna(subset=['description'], inplace = True)
    
    print(f"{nans} rows droped in colum {column}")
    
    return

In [80]:
drop_na(df, "description")

27 rows droped in colum description


In [81]:
df.shape

(988, 6)

In [82]:
# Vamos a ver que datos podemos encontrar en la columna 3:
not_nan_3 = df[df[3].isna() == False]

not_nan_3

,description,units,price,date,order_num,3
2,Leche entera Pascual,NaN,1.0,16/12/21,10822769,"5,64 €"
3,Leche desnatada Pascual,NaN,4.0,16/12/21,10822769,"22,56 €"
4,Mantequilla con sal Hacendado,NaN,1.0,16/12/21,10822769,"1,65 €"
5,Café en cápsula Colombia L'Or Espresso,NaN,1.0,16/12/21,10822769,"3,75 €"
6,Café en cápsula sontuoso L'Or Espresso,NaN,1.0,16/12/21,10822769,"3,75 €"
...,...,...,...,...,...,...
30,Leche desnatada Pascual,NaN,5.0,07/10/22,12804912,"32,70 €"
31,11 Panecillos,NaN,1.0,07/10/22,12804912,"1,10 €"
32,Patatas,NaN,1.0,07/10/22,12804912,"5,49 €"
33,Queso azul hojas de vaca,NaN,1.0,07/10/22,12804912,"1,91 €"


In [83]:
#Eliminamos estas filas (luego las vamos a volver a incluir)
df = df[df[3].isna() == True]
df.shape

(906, 6)

In [84]:
# Vemos que es como si se hubieram desplazado, lo reognanizamos: 
not_nan_correct = not_nan_3
not_nan_correct["units"] = not_nan_3["price"]
not_nan_correct["price"] = not_nan_3[3]
not_nan_correct[3] = np.nan

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_17572/4124051874.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_nan_correct["units"] = not_nan_3["price"]
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_17572/4124051874.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_nan_correct["price"] = not_nan_3[3]
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_17572/4124051874.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [85]:
not_nan_correct.sample()
df = pd.concat([df, not_nan_correct])

In [86]:
df.shape # Se mantien el numpero de columnas por lo que lo hemos hecho bien

(988, 6)

In [87]:
df = df.drop(3, axis=1)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 988 entries, 0 to 34
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  988 non-null    object
 1   units        939 non-null    object
 2   price        935 non-null    object
 3   date         988 non-null    object
 4   order_num    988 non-null    object
dtypes: object(5)
memory usage: 46.3+ KB


In [89]:
df.dropna(subset=['price'], inplace=True)

In [90]:
df.dropna(subset=['units'], inplace=True)

In [93]:
df["price_eu"] = df["price"].str.replace("€","").str.replace(",",".").str.strip().astype(float)

In [94]:
df = df[df['price_eu'] != 0]

In [95]:
df

,description,units,price,date,order_num,price_eu
0,Detergente ropa prendas delicadas Bosque Verde...,1,"1,69 €",10/12/20,7119732,1.69
1,Raclette queso Entremont lonchas,1,"3,85 €",10/12/20,7119732,3.85
2,Refresco Coca-Cola,1,"6,96 €",10/12/20,7119732,6.96
3,Queso blando blanco y azul Bergader petit blu,1,"1,85 €",10/12/20,7119732,1.85
4,Queso brie con trufa Castello,1,"2,10 €",10/12/20,7119732,2.10
...,...,...,...,...,...,...
30,Leche desnatada Pascual,5.0,"32,70 €",07/10/22,12804912,32.70
31,11 Panecillos,1.0,"1,10 €",07/10/22,12804912,1.10
32,Patatas,1.0,"5,49 €",07/10/22,12804912,5.49
33,Queso azul hojas de vaca,1.0,"1,91 €",07/10/22,12804912,1.91


In [98]:
df = df[df["units"] != "0,31 kg"]

In [102]:
df["units"] = df["units"].astype(int)

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_17572/3243935682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["units"] = df["units"].astype(int)


In [104]:
df["unit_price"] = df["price_eu"]/df["units"]

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_17572/3943752466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["unit_price"] = df["price_eu"]/df["units"]


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 933 entries, 0 to 34
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   description  933 non-null    object 
 1   units        933 non-null    int64  
 2   price        933 non-null    object 
 3   date         933 non-null    object 
 4   order_num    933 non-null    object 
 5   price_eu     933 non-null    float64
 6   unit_price   933 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 58.3+ KB


In [106]:
df

,description,units,price,date,order_num,price_eu,unit_price
0,Detergente ropa prendas delicadas Bosque Verde...,1,"1,69 €",10/12/20,7119732,1.69,1.69
1,Raclette queso Entremont lonchas,1,"3,85 €",10/12/20,7119732,3.85,3.85
2,Refresco Coca-Cola,1,"6,96 €",10/12/20,7119732,6.96,6.96
3,Queso blando blanco y azul Bergader petit blu,1,"1,85 €",10/12/20,7119732,1.85,1.85
4,Queso brie con trufa Castello,1,"2,10 €",10/12/20,7119732,2.10,2.10
...,...,...,...,...,...,...,...
30,Leche desnatada Pascual,5,"32,70 €",07/10/22,12804912,32.70,6.54
31,11 Panecillos,1,"1,10 €",07/10/22,12804912,1.10,1.10
32,Patatas,1,"5,49 €",07/10/22,12804912,5.49,5.49
33,Queso azul hojas de vaca,1,"1,91 €",07/10/22,12804912,1.91,1.91


In [107]:
df = df[["description", "units", "price_eu", "unit_price", "date", "order_num"]]

In [108]:
df

,description,units,price_eu,unit_price,date,order_num
0,Detergente ropa prendas delicadas Bosque Verde...,1,1.69,1.69,10/12/20,7119732
1,Raclette queso Entremont lonchas,1,3.85,3.85,10/12/20,7119732
2,Refresco Coca-Cola,1,6.96,6.96,10/12/20,7119732
3,Queso blando blanco y azul Bergader petit blu,1,1.85,1.85,10/12/20,7119732
4,Queso brie con trufa Castello,1,2.10,2.10,10/12/20,7119732
...,...,...,...,...,...,...
30,Leche desnatada Pascual,5,32.70,6.54,07/10/22,12804912
31,11 Panecillos,1,1.10,1.10,07/10/22,12804912
32,Patatas,1,5.49,5.49,07/10/22,12804912
33,Queso azul hojas de vaca,1,1.91,1.91,07/10/22,12804912


In [109]:
df['date'] = pd.to_datetime(df['date'])

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_17572/3532345252.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_17572/3532345252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


In [110]:
export_to_csv(df,"../../data/history-pruchase-more")

DataFrame successfully exported to ../../data/history-pruchase-more.csv


In [111]:
df

,description,units,price_eu,unit_price,date,order_num
0,Detergente ropa prendas delicadas Bosque Verde...,1,1.69,1.69,2020-10-12,7119732
1,Raclette queso Entremont lonchas,1,3.85,3.85,2020-10-12,7119732
2,Refresco Coca-Cola,1,6.96,6.96,2020-10-12,7119732
3,Queso blando blanco y azul Bergader petit blu,1,1.85,1.85,2020-10-12,7119732
4,Queso brie con trufa Castello,1,2.10,2.10,2020-10-12,7119732
...,...,...,...,...,...,...
30,Leche desnatada Pascual,5,32.70,6.54,2022-07-10,12804912
31,11 Panecillos,1,1.10,1.10,2022-07-10,12804912
32,Patatas,1,5.49,5.49,2022-07-10,12804912
33,Queso azul hojas de vaca,1,1.91,1.91,2022-07-10,12804912
